In [1]:
import os,sys
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import pennylane as qml
import torch
import numpy as np

In [3]:
import time

In [4]:
import builtins

In [5]:
print(qml.__version__)

0.8.0-dev


In [6]:
#builtins.dev_qubit = qml.device("projectq.simulator",wires=6,shots=100)
#builtins.dev_qubit  = qml.device('qiskit.aer', wires=6,shots=100)
builtins.dev_qubit  = qml.device('default.qubit', wires=8)

In [7]:
from modules.QCircuitNet import QCircuitNet

In [8]:
x=torch.ones(8,2)
y=torch.ones(8)
phi=torch.ones(12)
phi.requires_grad=True

In [9]:
from modules.circ_4datapoints import circuit

In [10]:
all_wires=list(range(8))

In [11]:
all_wires

[0, 1, 2, 3, 4, 5, 6, 7]

In [12]:
circuit(phi,Xdata=x,Y=y)

tensor([ 0.7500, -0.5000], dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)

In [9]:
my_torch_circ=QCircuitNet()

Parameter containing:
tensor([0.2312, 0.8450], requires_grad=True)


In [10]:
out=my_torch_circ(x,y)

In [11]:
crit=torch.nn.MSELoss()

In [12]:
crit(out,y[-1])

tensor(4., grad_fn=<MseLossBackward>)

In [13]:
from utils.data_handling import simple_np_ds

In [14]:
ds=simple_np_ds("../data/moons.npz",val_split=0.2,test_split=0.2)

In [15]:
len(ds)

1000

In [16]:
ds[2]

(array([-0.24548973,  0.668868  ], dtype=float32), -1)

In [17]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [18]:
train_dldr=DataLoader(ds,
                      batch_size=3,
                      shuffle=False,
                      sampler=SubsetRandomSampler(ds.train_indices),
                      num_workers=3,
                      drop_last=True)

In [19]:
train_iter=iter(train_dldr)

In [36]:
tr_data = next(train_iter)

In [37]:
print(tr_data)

[tensor([[-0.0529,  0.3720],
        [-0.5403,  0.2575],
        [ 0.3323,  0.8629]]), tensor([ 1, -1, -1])]


In [38]:
data=tr_data[0]

In [39]:
data

tensor([[-0.0529,  0.3720],
        [-0.5403,  0.2575],
        [ 0.3323,  0.8629]])

In [40]:
labels=tr_data[1]

In [41]:
labels

tensor([ 1, -1, -1])

In [42]:
data_adj=torch.zeros(data.shape[0]+1,data.shape[1])

In [43]:
data_adj[:-1]=data[:]
data_adj[-1]=data[-1]

In [44]:
data_adj

tensor([[-0.0529,  0.3720],
        [-0.5403,  0.2575],
        [ 0.3323,  0.8629],
        [ 0.3323,  0.8629]])

In [45]:
labels_adj=torch.zeros(labels.shape[0]+1)

In [46]:
labels_adj[:-1]=labels[:]
labels_adj[-1]=labels[-1]

In [47]:
labels_adj

tensor([ 1., -1., -1., -1.])

In [72]:
labels.dtype

torch.int64

In [48]:
model_out=my_torch_circ(data_adj,labels_adj)

In [49]:
model_out

tensor(0.2208, grad_fn=<MulBackward0>)

In [50]:
torch.where(model_out>0.0,torch.tensor([1.0]),torch.tensor([-1.0]))

tensor([1.])

In [51]:
prediction = torch.where(model_out>0.0,torch.tensor([1.0]),torch.tensor([-1.0]))

In [52]:
val_dldr=DataLoader(ds,
                    batch_size=1,
                    shuffle=False,
                    sampler=SubsetRandomSampler(ds.val_indices),
                    num_workers=3,
                    drop_last=True)
val_iter=iter(val_dldr)

In [53]:
train_eval_dldr=DataLoader(ds,
                           batch_size=2,
                           shuffle=False,
                           sampler=SubsetRandomSampler(ds.train_indices),
                           num_workers=3,
                           drop_last=True)
train_eval_iter=iter(train_eval_dldr)

In [55]:
val_data_label = next(val_iter)
val_data=val_data_label[0]
val_label=val_data_label[1]

In [56]:
val_data

tensor([[-0.3328,  1.0557]])

In [57]:
val_label

tensor([-1])

In [58]:
ens_train_data = next(train_eval_iter)

In [59]:
ens_data=ens_train_data[0]
ens_label=ens_train_data[1]

In [60]:
ens_data

tensor([[0.7985, 0.5045],
        [0.7486, 0.8989]])

In [61]:
ens_label

tensor([-1, -1])

In [64]:
full_data=torch.cat((ens_data,val_data))

In [65]:
full_data

tensor([[ 0.7985,  0.5045],
        [ 0.7486,  0.8989],
        [-0.3328,  1.0557]])

In [68]:
full_label=torch.cat((ens_label,val_label))

In [69]:
full_label

tensor([-1, -1, -1])